Connect to Postgres Table. This notebook already contains three lines of code so you do not have to enter them. Run these three lines. The first line imports the SQLContext module, which is needed access SQL databases in Spark

In [1]:
from pyspark.sql import SQLContext

In [3]:
sqlsc = SQLContext(sc)

The third line creates a new Spark DataFrame in the variable df for the Postgres table gameclicks

In [4]:
df = sqlsc.read.format("jdbc") \
  .option("url", "jdbc:postgresql://localhost/cloudera?user=cloudera") \
  .option("dbtable", "gameclicks") \
  .load()

The format("jdbc") says that the source of the DataFrame will be using a Java database connection, the url option is the URL connection string to access the Postgres database, and the dbtable option specifies the gameclicks table

View Spark DataFrame schema and count rows. We can call the printSchema() method to view the schema of the DataFrame

In [6]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = false)
 |-- clickid: integer (nullable = false)
 |-- userid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- ishit: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- teamlevel: integer (nullable = false)



The descriptions list the name and data type of each column

We can also call the count() method to count the number of rows in the DataFrame

In [9]:
df.count()

755806

View contents of DataFrame. We can call the show() methowd to view the contents of the DataFrame. The argument specifies how many rows to display:

In [10]:
df.show(5)

+--------------------+-------+------+-------------+-----+------+---------+
|           timestamp|clickid|userid|usersessionid|ishit|teamid|teamlevel|
+--------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:...|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:...|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:...|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:...|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:...|     22|  1362|         5739|    0|    13|        1|
+--------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



Filter columns in DataFrame. We can filter for one or more columns by calling the select() method:

In [11]:
df.select("userid", "teamlevel",).show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



 Filter rows based on criteria using filter()

In [12]:
df.filter(df["teamlevel"] > 1).select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1513|        2|
|   868|        2|
|  1453|        2|
|  1282|        2|
|  1473|        2|
+------+---------+
only showing top 5 rows



The arguments of filter are a column to fix the criteria ("teamlevel") and the condition (>1). The remainder is just select and show as previously seen.

Group by a column and count. The groupBy() method groups the values of column(s). The ishit column only has values 0 and 1. We can caluclate how many times each occurs by grouping the ishit column and counting the result.

In [14]:
df.groupBy("ishit").count().show()

+-----+------+
|ishit| count|
+-----+------+
|    0|672423|
|    1| 83383|
+-----+------+



Calculate average and sum. Aggregate operations can be performed on columns of DataFrames. First, let's import the Python libraries for the aggregate operations. Next we calculate the average and total values by calling mean() and sum() methods.

In [16]:
from pyspark.sql.functions import *
df.select(mean('ishit'), sum('ishit')).show()

+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



Join two DataFrames. We can merge or join two DataFrames on a single column. First lets create a DataFrame for the adclicks table in the postgres database by copying the third cell in this notebook and changing gameclicks to adclicks and storing the result in df2.

In [19]:
df2 = sqlsc.read.format("jdbc")
.option("url", "jdbc:postgresql://localhost/cloudera?user=cloudera")
.option("dbtable", "adclicks")
.load()

In [20]:
df2.printSchema()

root
 |-- timestamp: timestamp (nullable = false)
 |-- txid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- userid: integer (nullable = false)
 |-- adid: integer (nullable = false)
 |-- adcategory: string (nullable = false)



We can see that df2 has also a column named "userid". We will merge the dataframes with this variable.

In [21]:
merge = df.join(df2, "userid")

In [22]:
merge.printSchema()

root
 |-- userid: integer (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- clickid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- ishit: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- teamlevel: integer (nullable = false)
 |-- timestamp: timestamp (nullable = false)
 |-- txid: integer (nullable = false)
 |-- usersessionid: integer (nullable = false)
 |-- teamid: integer (nullable = false)
 |-- adid: integer (nullable = false)
 |-- adcategory: string (nullable = false)



Merge has all the columns from both adclicks and gameclicks

In [23]:
merge.show(5)

+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+----------+
|userid|           timestamp|clickid|usersessionid|ishit|teamid|teamlevel|           timestamp| txid|usersessionid|teamid|adid|adcategory|
+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+----------+
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 01:40:...|23669|        23626|   142|  27|     games|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 09:24:...|24122|        23626|   142|   4|     games|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 17:21:...|24659|        23626|   142|  22| computers|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 23:34:...|25076|        23626|   142|  21|    movies|
|   231|2016-06-08 00:45:..